In [1]:
# Data handling dependencies
import pandas as pd
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Custom Libraries
import sys
sys.path.append(r'C:\Users\Kamogelo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\Scripts\surprise.exe')
from utils.data_loader import load_movie_titles
from recommenders.collaborative_based import collab_model
from recommenders.content_based import content_model


In [2]:

# Data Loading
title_list = load_movie_titles('resources/data/movies.csv')
movies_df =  pd.read_csv('resources/data/movies.csv', index_col='movieId')

## Content-based filtering

In [3]:

def data_preprocessing(subset_size):
    """Prepare data for use within Content filtering algorithm.
    Pandas Dataframe
        Subset of movies selected for content-based filtering.

    """
    # Split genre data into individual words.
    movies['keyWords'] = movies['genres'].str.replace('|', ' ')
    # Subset of the data
    movies_subset = movies[:subset_size]
    return movies_subset
 
def content_model(movie_list,top_n=10): 
    """Performs Content filtering based upon a list of movies supplied
       by the app user.
    """
    # Initializing the empty list of recommended movies
    data = data_preprocessing(2000)
    # Instantiating and generating the count matrix
    count_vec = CountVectorizer()
    count_matrix = count_vec.fit_transform(data['keyWords'])
    indices = pd.Series(data['title'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    cosine_sim = pd.DataFrame(cosine_sim, index = data.index, columns = data.index)
    # Getting the index of the movie that matches the title
    idx_1 = indices[indices == movie_list[0]].index[0]
    idx_2 = indices[indices == movie_list[1]].index[0]
    idx_3 = indices[indices == movie_list[2]].index[0]
    # Creating a Series with the similarity scores in descending order
    rank_1 = cosine_sim[idx_1]
    rank_2 = cosine_sim[idx_2]
    rank_3 = cosine_sim[idx_3]
    # Calculating the scores
    score_series_1 = pd.Series(rank_1).sort_values(ascending = False)
    score_series_2 = pd.Series(rank_2).sort_values(ascending = False)
    score_series_3 = pd.Series(rank_3).sort_values(ascending = False)
    # Getting the indexes of the 10 most similar movies
    listings = pd.concat([score_series_1, score_series_2, score_series_3]).sort_values(ascending=False)
    # Store movie names
    recommended_movies = []
    # Appending the names of movies
    top_50_indexes = list(listings.iloc[1:50].index)
    # Removing chosen movies
    top_indexes = np.setdiff1d(top_50_indexes,[idx_1,idx_2,idx_3])
    for i in top_indexes[:top_n]:
        recommended_movies.append(list(movies['title'])[i])
    return recommended_movies

In [4]:
movies = movies_df.dropna()
movie_list = [title_list[1],title_list[2],title_list[3]]
content_model(movie_list,top_n=10)

['Dracula: Dead and Loving It (1995)',
 'Lamerica (1994)',
 'Confessional, The (Confessionnal, Le) (1995)',
 'Eye for an Eye (1996)',
 'In the Bleak Midwinter (1995)',
 "Jupiter's Wife (1994)",
 'Frankie Starlight (1995)',
 'Stars Fell on Henrietta, The (1995)',
 'Drop Zone (1994)',
 'Ed Wood (1994)']

## Collaborative-based filtering

In [5]:
import pandas as pd
import numpy as np
import pickle
import copy
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
# Importing data
movies_df = pd.read_csv('resources/data/movies.csv',sep = ',')
ratings_df = pd.read_csv('resources/data/ratings.csv')
ratings_df.drop(['timestamp'], axis=1,inplace=True)

# We make use of an SVD model trained on a subset of the MovieLens 10k dataset.
model=pickle.load(open('resources/models/SVD.pkl', 'rb'))

In [7]:

# We make use of an SVD model trained on a subset of the MovieLens 10k dataset.
model=pickle.load(open('resources/models/SVD.pkl', 'rb'))

def prediction_item(item_id):
    """Map a given favourite movie to users within the
       MovieLens dataset with the same preference.

    Parameters
    ----------
    item_id : int
        A MovieLens Movie ID.

    Returns
    -------
    list
        User IDs of users with similar high ratings for the given movie.

    """
    # Data preprosessing
    reader = Reader(rating_scale=(0, 5))
    load_df = Dataset.load_from_df(ratings_df,reader)
    a_train = load_df.build_full_trainset()

    predictions = []
    for ui in a_train.all_users():
        predictions.append(model.predict(iid=item_id,uid=ui, verbose = False))
    return predictions

def pred_movies(movie_list):
    """Maps the given favourite movies selected within the app to corresponding
    users within the MovieLens dataset.

    Parameters
    ----------
    movie_list : list
        Three favourite movies selected by the app user.

    Returns
    -------
    list
        User-ID's of users with similar high ratings for each movie.

    """
    # Store the id of users
    id_store=[]
    # For each movie selected by a user of the app,
    # predict a corresponding user within the dataset with the highest rating
    for i in movie_list:
        predictions = prediction_item(item_id = i)
        predictions.sort(key=lambda x: x.est, reverse=True)
        # Take the top 10 user id's from each movie with highest rankings
        for pred in predictions[:10]:
            id_store.append(pred.uid)
    # Return a list of user id's
    return id_store

# !! DO NOT CHANGE THIS FUNCTION SIGNATURE !!
# You are, however, encouraged to change its content.  
def collab_model(movie_list, top_n=10):
    """Performs Collaborative filtering based upon a list of movies supplied
       by the app user.

    Parameters
    ----------
    movie_list : list (str)
        Favorite movies chosen by the app user.
    top_n : type
        Number of top recommendations to return to the user.

    Returns
    -------
    list (str)
        Titles of the top-n movie recommendations to the user.
    """
    
    # Retrieving indices of movies
    indices = pd.Series(movies_df['title'])
    
    # Getting movie IDs based on the provided list
    movie_ids = pred_movies(movie_list)
    
    # Collecting ratings for users who rated the movies from the provided list
    df_init_users = ratings_df[ratings_df['userId'] == movie_ids[0]]
    
    # Concatenating ratings for each movie in the list
    for i in movie_ids[1:]:  # Loop starts from the second movie_id
        df_init_users = pd.concat([df_init_users, ratings_df[ratings_df['userId'] == i]])

    # Getting the cosine similarity matrix
    cosine_sim = cosine_similarity(df_init_users.drop(columns=['userId']), df_init_users.drop(columns=['userId']))
    
    # Retrieving indices of movies from the provided list
    idx_1 = indices[indices == movie_list[0]].index[0]
    idx_2 = indices[indices == movie_list[1]].index[0]
    idx_3 = indices[indices == movie_list[2]].index[0]
    
    # Creating a Series with the similarity scores in descending order
    rank_1 = cosine_sim[idx_1]
    rank_2 = cosine_sim[idx_2]
    rank_3 = cosine_sim[idx_3]
    
    # Calculating the scores
    score_series_1 = pd.Series(rank_1).sort_values(ascending=False)
    score_series_2 = pd.Series(rank_2).sort_values(ascending=False)
    score_series_3 = pd.Series(rank_3).sort_values(ascending=False)
    
    # Appending the names of movies
    listings = pd.concat([score_series_1, score_series_2, score_series_3]).sort_values(ascending=False)
    
    recommended_movies = []
    
    # Choose top 50
    top_50_indexes = list(listings.iloc[1:50].index)
    # Removing chosen movies
    top_indexes = np.setdiff1d(top_50_indexes, [idx_1, idx_2, idx_3])
    
    # Getting the titles of the top-n recommended movies
    for i in top_indexes[:top_n]:
        recommended_movies.append(list(movies_df['title'])[i])
    
    return recommended_movies


In [8]:

movie_list = [title_list[1],title_list[2],title_list[3]]
collab_model(movie_list,top_n=10)

['Juror, The (1996)',
 'White Balloon, The (Badkonake sefid) (1995)',
 'Under Siege 2: Dark Territory (1995)',
 'Burnt by the Sun (Utomlyonnye solntsem) (1994)',
 "Muriel's Wedding (1994)",
 'Baby-Sitters Club, The (1995)',
 'Ace Ventura: Pet Detective (1994)',
 'Higher Learning (1995)',
 'I Love Trouble (1994)',
 'Tough and Deadly (1995)']

## API

In [9]:
import requests
from IPython.display import display, HTML

def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=c7ec19ffdd3279641fb606d19ceb9bb1&language=en-US".format(movie_id)
    data=requests.get(url)
    data=data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/"+poster_path
    return full_path

In [10]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movie_id = 862

In [32]:
from IPython.display import Image, display

image_url = fetch_poster(movie_id)
display(Image(url=image_url))


In [33]:
from IPython.display import IFrame

# Define the URL
url = "https://www.imdb.com/title/tt0113497/"

# Display the content in an iframe
IFrame(url, width=800, height=600)


In [ ]:
https://www.imdb.com/title/tt0(113497)/